In [ ]:
import sympy
from sympy import symbols, diff, ln
from sympy.codegen.cfunctions import log2
import functools
import matplotlib.pyplot as plt

x0=symbols("x0")
x1=symbols("x1")

def call(f, x):
    return f(x)

def mapv(a, b):
    return list(map(a, map(lambda x: 1.0 * x, b)))

def mapv_indexed(f, coll):
    return list(map(lambda x: [x, f(x)], map(lambda x: 1.0 * x, coll)))

class PipeAsCall(object):
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        return self.func(other)
    def __call__(self, x):
        return self.func(x)

class Transducer(PipeAsCall):
  pass

def partial(*args):
    return PipeAsCall(
        functools.partial(*args))

def D(f):
  var = symbols("var")
  return PipeAsCall(sympy.lambdify([var], diff(f(var), var), "sympy"))

def rnd(expr):
     return expr.xreplace({n: round(n, 3) for n in expr.atoms(sympy.Number)})

def plot(f, x):
    if (type(f)==str) or (type(f)==sympy.core.numbers.Float):
       return "That's not something you can plot"
    else:
       plt.plot(x, mapv(f, x), "o")

def plotting(f):
    if type(f) == Transducer:
        return partial(lambda f, coll: plt.plot(list(map(lambda x: x[0],f(coll))),list(map(lambda x: x[1],f(coll))), "o"), f)
    else:
        return partial(lambda f, x: plt.plot(x, mapv(f, x), "o"), f)

isPositive = lambda n: True if n>0 else False

def smaller(x):
    return lambda n: True if n<x else False

def filtering(f):
    return Transducer(functools.partial(lambda f,coll: list(filter(f, coll)), f))

def mapping(f):
    return Transducer(functools.partial(mapv_indexed, f))

def compose(f, g):
    return Transducer(lambda x: f(g(x)))

In [ ]:
import numpy as np
import json
import inspect


# https://muted.io/note-frequencies/
data = np.genfromtxt('notes.txt', delimiter='\t', dtype=str)
row_names = data[:, 0].tolist()
numerical_data = np.matrix.transpose(data[:, 1:]).astype(float).tolist()

nofreq = [item
    for subl in
      [list(map(lambda n,f: [n, idx, f], row_names, fs))
        for idx, fs in enumerate(numerical_data)]
   for item in subl]

frequencies = [n[2] for n in nofreq]

In [ ]:
plot(log2, [1, 2, 4, 8])

In [ ]:
plot("log2(x / 220) * 7", [1, 2, 4, 8])

In [ ]:
def Poe(intersect, steps, x):
    return log2(x/intersect) * steps

Poe(220, 7, 440)

In [ ]:
def log2b(intersect, steps):
    return partial(Poe, intersect, steps)

In [ ]:
plot(Poe(220, 7, 440), [1, 2, 4, 8])

In [ ]:
plot(partial(Poe, 220, 7), [1, 2, 4, 8])

In [ ]:
plot(log2b(220, 7), [1, 2, 4, 8])

In [ ]:
call(log2b(220, 7), 8)

In [ ]:
call(log2b(220, 7), x0)

In [ ]:
call(log2b(1, 0.7), x1)

In [ ]:
call(D(log2b(1, 0.7)), x0).evalf()

In [ ]:
call(D(ln), x0)

In [ ]:
(D(ln) | x0)

In [ ]:
(plotting(ln) | frequencies)

In [ ]:
(plotting(ln) |
  (filtering(smaller(900)) |
    frequencies))

In [ ]:
(plotting
  (compose
    (mapping(ln), 
     filtering(smaller(900)))) |
  frequencies)

In [ ]:
(plotting
  (compose
    (mapping(D(ln)), 
     filtering(smaller(900)))) |
  frequencies)